In [2]:

def fetch_field_data(cursor, field_name):
    """
    Fetches tender_id and specified field data from the database.

    Parameters:
    - cursor: A database cursor object.
    - field_name: The name of the field to fetch data from.

    Returns:
    A list of dictionaries, each containing 'tender_id' and the specified field data.
    """
    

    
    # 构造SQL查询语句
    query = f"SELECT tender_id, {field_name} FROM tender_detail"
    
    # 执行查询
    try:
        cursor.execute(query)
        # 获取所有记录
        rows = cursor.fetchall()
        # 转换记录为JSON格式
        data = [{"tender_id": row[0], field_name: row[1]} for row in rows]
        return data
    except Exception as e:
        print(f"Error fetching data: {e}")
        return []


In [186]:
tool_for_chief_supervisor = [{
    "type": "function",
    "function": {
        "name": "insert_chief_supervisor_req",
        "description": "将注册资格证书专业ID(注意，如果有两个专业的要求，则分别填入两个专业id arguments)、职称等级ID、最低学历要求ID、总监相关业绩要求和总监同时任职项目数量限制等信息插入到数据库中。",
        "parameters": {
        "type": "object",
        "properties": {
            "tender_id": {
            "description": "tender_id",
            "type": "string"
            },
            "专业ID一": {
            "description": "注册资格证书专业ID:房屋建筑工程:1; 冶炼工程:2; 矿山工程:3; 化工石油工程:4; 水利水电工程:5; 电力工程:6; 农林工程:7; 铁路工程:8; 公路工程:9; 港口与航道工程:10; 航天航空工程:11; 通信工程:12; 市政公用工程:13; 机电安装工程:14; 道路与桥梁:15; 隧道工程:16; 公路机电工程:17; 港口工程:18; 航道工程:19; 水运机电工程:20; 水利工程施工监理:21; 水土保持工程施工监理:22; 机电及金属结构设备制造监理:23; 水利工程建设环境保护监理:24;",
            "type": "string",
            "enum": ["Null","1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24"]
            },
            "专业ID二": {
            "description": "（可选）注册资格证书专业ID:房屋建筑工程:1; 冶炼工程:2; 矿山工程:3; 化工石油工程:4; 水利水电工程:5; 电力工程:6; 农林工程:7; 铁路工程:8; 公路工程:9; 港口与航道工程:10; 航天航空工程:11; 通信工程:12; 市政公用工程:13; 机电安装工程:14; 道路与桥梁:15; 隧道工程:16; 公路机电工程:17; 港口工程:18; 航道工程:19; 水运机电工程:20; 水利工程施工监理:21; 水土保持工程施工监理:22; 机电及金属结构设备制造监理:23; 水利工程建设环境保护监理:24;",
            "type": "string",
            "enum": ["Null","1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12", "13", "14", "15", "16", "17", "18", "19", "20", "21", "22", "23", "24"]
            },
            "职称等级ID": {
            "description": "职称等级ID:教授级高级工程师（正高级）:1 ;高级工程师（副高级）:2;工程师（中级）:3;助理工程师（初级）:4",
            "type": "string",
            "enum": ["Null","1", "2", "3", "4"]
            },
            "学历ID": {
            "description": "最低学历要求ID:中专/高中:1;专科:2,本科:3;硕士研究生:4;博士研究生:5;其他:Null",
            "type": "string",
            "enum": ["Null","1", "2", "3", "4", "5"]
            },
            "相关业绩要求": {
            "description": "总监相关业绩要求，例如‘至少担任过2项类似工程的监理负责人’。没有要求则是Null。",
            "type": "number"
            },
            "同时任职项目限制": {
            "description": "总监同时任职项目数量限制，例如‘在任职期间能参与的其他在施项目不得超过2个’。没有要求则是Null。",
            "type": "number"
            }
        },
        "required": ["tender_id","专业ID一","专业ID二", "职称等级ID", "学历ID", "相关业绩要求", "同时任职项目数"]
        }
    }
    }]



In [216]:
tool_for_price = [{
    "type": "function",
    "function": {
        "name": "insert_chief_supervisor_req",
        "description": """将招标价格（单位是元），插入到数据库中;若提供的信息不足以明确具体的价格（单位为元），则输出“Null”;
        例如：
        1. 明确万元单位：“119.69万元”，要转化单位为元，并格式化为小数点后两位：
        - argument输入：“1,196,900.00”
        2. 未提供单位，需忽略：“940.300479”
        - argument输入：“Null”
        3. 包含非价格信息，忽略：“监理费费率不得大于0.80”
        - argument输入：“Null”
        """,
        "parameters": {
        "type": "object",
        "properties": {
            "tender_id": {
            "description": "tender_id",
            "type": "string"
            },
            "招标价格": {
            "description": "当价格以万元为单位时，转换成元，并格式化为小数点后两位，例如“100万元”转化成“1,000,000.00”",
            "type": "number",
            },
        },
        "required": ["tender_id","招标价格"]
        }
    }
    }]



In [313]:
tool_for_price = [{
    "type": "function",
    "function": {
        "name": "insert",
        "description": """
        将提取并转换后的招标价格数值（单位为元）插入到数据表中。价格必须是完整的，即包含明确的数值和单位。
        """,
        "parameters": {
        "type": "object",
        "properties": {
            "tender_id": {
            "description": "tender_id",
            "type": "string"
            },
            "招标价格": {
            "description": "经过转换的招标价格数值，单位为元。价格必须是完整的。",
            "type": "number",
            },
        },
        "required": ["tender_id","招标价格"]
        }
    }
    },
    {
    "type": "function",
    "function": {
        "name": "lable_null",
        "description": """
        当价格信息不完整时，使用此函数标注相应的招标公告的价格数据为不完整。价格信息不完整定义为没有明确数值和单位。
        """,
        "parameters": {
        "type": "object",
        "properties": {
            "tender_id": {
            "description": "tender_id",
            "type": "string"
            },
        },
        "required": ["tender_id"]
        }
    }
    }
    ]



In [376]:
from zhipuai import ZhipuAI

client = ZhipuAI(api_key="e6af334544b37a85e83900a9152eb9a0.GzjnBTUhPOhDDJhx")

def proofread_values(values,tools,prompt):

    # 清空对话
    messages = []
# 
    # messages.append({"role": "system", "content": """
    #                     在招标公告中，根据insert函数要求的arguments要求，提取具体信息（可根据语意进行合理推断），再通过insert函数将数据插入到数据库中。arguments里如果没有相应的信息，则输出“Null”。
    #                     """})

    messages.append(prompt)
    messages.append({"role": "user", "content": "以下是招标公告中相关字段值："})

    messages.append({"role": "user", "content": f"{values}"})

    response = client.chat.completions.create(
        model="glm-4",  # 填写需要调用的模型名称 model="glm-4"  or "glm-3-turbo"
        messages=messages,
        tools=tools,
        top_p=0.7,
        temperature=0.8,
        max_tokens=4096,
    )
    return response
    # return response.choices[0].message.tool_calls[0].function.arguments

In [188]:
from database_util import connect_db
db = connect_db()
cursor = db.cursor()

# 示例使用
# 假设您想提取 'chief_supervisor_qualification_requirement' 字段的数据
field_name = 'chief_supervisor_qualification_requirement'
data = fetch_field_data(cursor, field_name)
print(data)


[{'tender_id': 1, 'chief_supervisor_qualification_requirement': '路桥相关专业高级工程师，至少担任过2项类似工程土建工程施工监理的监理负责人。'}, {'tender_id': 3, 'chief_supervisor_qualification_requirement': '具备国家注册监理工程师执业资格（市政公用工程专业）'}, {'tender_id': 4, 'chief_supervisor_qualification_requirement': '具备国家注册监理工程师执业资格（市政公用工程专业）'}, {'tender_id': 6, 'chief_supervisor_qualification_requirement': '拟派本工程总监理工程师具备市政公用工程专业和建筑工程专业的注册监理工程师证书'}, {'tender_id': 7, 'chief_supervisor_qualification_requirement': '具有建设部2006年4月1日后颁发的中华人民共和国注册监理工程师注册执业证书，专业为市政公用工程，注册执业单位为投标人本公司，且在有效期内'}, {'tender_id': 8, 'chief_supervisor_qualification_requirement': '1、需登记在东莞市公共资源交易企业库中；2、具备建设部核发的《中华人民共和国注册监理工程师注册执业证书》，上述证书上注明的注册专业为房屋建筑工程专业，上述证书上注明的注册执业单位（或工作单位）为投标人本单位；3、项目总监理工程师不得同时在除本工程外的其他任何工程项目中任职。'}, {'tender_id': 9, 'chief_supervisor_qualification_requirement': '具有市政公用工程专业国家注册监理工程师资格'}, {'tender_id': 10, 'chief_supervisor_qualification_requirement': '房屋建筑工程'}, {'tender_id': 11, 'chief_supervisor_qualification_requirement': '项目总监必须具备国家注册监理工程师执业资格'}, {'ten

In [314]:
price_prompt = {"role": "system", "content": """
我需要GLM智能体执行以下任务：

1. 接收输入的招标公告文本，其中包含与价格相关的字段信息。
2. 准确提取文本中的价格信息，并确保价格包含明确的数值和单位。
3. 将提取的价格信息转换为以元为单位的数值。例如，如果输入为"100万元"，则应转换为1,000,000.00元。
4. 调用预先编写好的insert函数，将转换后的价格数值插入到数据表中。
5. 如果价格信息不完整，即没有明确数值或单位，或者包含非价格信息，例如"905123"或"监理费费率不得大于0.80"，则调用label_null函数，并标注该价格信息不完整。

价格完整性判断标准：
- 价格必须包含明确的数值和单位。
- 如果价格只包含数字，没有单位，则无法确定具体金额，应视为价格信息不完整。
- 如果价格信息中包含除数值和单位外的其他文字，如描述性或限制性语句，也应视为价格信息不完整。

请确保智能体能够正确处理上述所有情况，并在处理过程中保持高效和准确。
"""}

In [388]:
price_prompt = {"role": "system", "content": """
AI代理，从提供的数据中提取招标价格，仅输出数值，例如"180100"。

并遵循以下规则进行转换：
1. 当价格以万元为单位时，转换成元的数值，例如“100万元”转化成“1000000”。
2. 若提供的信息没有单位(比如，仅仅 “420” 或 “186023"，或者不足以明确具体的价格，则输出“Null”。
3. 请注意，我们只关注服务的招标价格，不考虑其他类型的金额；务必在转化单位的时候，进行计算，以保证转化的数字绝对正确（不必展示过程，进需要输出要求的结果）


"""}

In [326]:
tool_for_price=[]

In [270]:
price_data = fetch_field_data(cursor, "maximum_bid_price")


In [360]:
price_data

[{'tender_id': 1, 'maximum_bid_price': '23691363'},
 {'tender_id': 3, 'maximum_bid_price': None},
 {'tender_id': 4, 'maximum_bid_price': '未明确说明'},
 {'tender_id': 6, 'maximum_bid_price': '115万元'},
 {'tender_id': 7, 'maximum_bid_price': '监理费经估算约3563800.00元'},
 {'tender_id': 8, 'maximum_bid_price': '监理服务收费系数不得大于0.80'},
 {'tender_id': 9, 'maximum_bid_price': '215.239374'},
 {'tender_id': 10, 'maximum_bid_price': '677.6万元'},
 {'tender_id': 11, 'maximum_bid_price': '277.6515 万元'},
 {'tender_id': 15, 'maximum_bid_price': '5976677.78元'},
 {'tender_id': 16, 'maximum_bid_price': '390000.00元'},
 {'tender_id': 17, 'maximum_bid_price': '239400.00元'},
 {'tender_id': 18, 'maximum_bid_price': '275200.00'},
 {'tender_id': 21, 'maximum_bid_price': '100%'},
 {'tender_id': 23, 'maximum_bid_price': '128.75 万元'},
 {'tender_id': 24, 'maximum_bid_price': '49.06万元'},
 {'tender_id': 25, 'maximum_bid_price': '871.45'},
 {'tender_id': 26, 'maximum_bid_price': '40万元（暂定价）'},
 {'tender_id': 27, 'maximum_bid_price': 

In [389]:
price_data[77]

{'tender_id': 107, 'maximum_bid_price': '2451300.000000'}

In [390]:
results = proofread_values(price_data[77],tool_for_price,price_prompt)

In [391]:
print(results.choices[0].message.content)

2451300


In [345]:
results.choices[0].message.tool_calls[0].function

TypeError: 'NoneType' object is not subscriptable

In [331]:
results.usage.model_dump()

{'prompt_tokens': 310, 'completion_tokens': 95, 'total_tokens': 405}

In [ ]:
def main_process(cursor):
    # 1. 从数据库获取所有招标详情
    tender_details = fetch_all_tender_details(cursor)

    # 遍历每一个招标记录
    for tender_detail in tender_details:
        tender_id = tender_detail['tender_id']
        extracted_infos = []

        # 2. 对于每一个字段，根据字段名称使用不同的提取策略
        # 假设 tender_detail 字典中已经包含了所有你感兴趣的字段及其值
        for field_name, field_value in tender_detail.items():
            if field_name == 'tender_id':
                continue  # 跳过 tender_id 字段，因为它不需要提取信息
            extracted_info = extract_info_with_tool(tender_id, field_name, field_value)
            if extracted_info:
                extracted_infos.append(extracted_info)
        
        # 3. 合并提取的信息
        combined_info = combine_info(tender_id, extracted_infos)

        # 4. 将合并后的信息插入到数据库中
        insert_into_database(cursor, combined_info)
    
    # 确保所有更改都被提交到数据库
    db.commit()
